In [1]:
import datetime
import time
current_time = datetime.datetime.now()
print(current_time)
timestamp = str(time.mktime(current_time.timetuple()))

2015-11-24 16:42:06.442659


In [3]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('bmh')
from pycalphad.fitting import build_pymc_model
from pycalphad import Database
import pymc
import numpy as np
import glob

input_database = Database('Al-Ni-start.tdb')
dataset_names = sorted(glob.glob('BCC_B2*.json'))

LB2ALVA_A = pymc.Normal('LB2ALVA_A', mu=0, tau=1./200000**2)
B2ALVA_A = pymc.Normal('B2ALVA_A', mu=0, tau=1./100000**2)
B2ALVA_B = pymc.Normal('B2ALVA_B', mu=0, tau=1./30**2)
LB2NIVA_A = pymc.Normal('LB2NIVA_A', mu=0, tau=1./100000**2)
LB2NIVA_B = pymc.Normal('LB2NIVA_B', mu=0, tau=1./30**2)
B2NIVA_A = pymc.Normal('B2NIVA_A', mu=0, tau=1./100000**2)
B2NIVA_B = pymc.Normal('B2NIVA_B', mu=0, tau=1./30**2)
LB2ALNI_A = pymc.Normal('LB2ALNI_A', mu=0, tau=1./100000**2)
LB2ALNI_B = pymc.Normal('LB2ALNI_B', mu=0, tau=1./30**2)
B2ALNI_A = pymc.Normal('B2ALNI_A', mu=0, tau=1./100000**2)
B2ALNI_B = pymc.Normal('B2ALNI_B', mu=0, tau=1./30**2)
params = [LB2ALVA_A, B2ALVA_A, B2ALVA_B, LB2NIVA_A,
          LB2NIVA_B, B2NIVA_A, B2NIVA_B, LB2ALNI_A,
          LB2ALNI_B, B2ALNI_A, B2ALNI_B]
%time mod = build_pymc_model(input_database, dataset_names, params)
M = pymc.MAP(mod)
%time M.fit()
MDL = pymc.MCMC(mod, db='hdf5', dbname='AlNi-BCC_B2-'+timestamp+'.hdf5', dbcomplevel=4, dbcomplib='bzip2')

CPU times: user 8.01 s, sys: 16 ms, total: 8.03 s
Wall time: 7.99 s
CPU times: user 7.31 s, sys: 0 ns, total: 7.31 s
Wall time: 7.3 s


In [ ]:
%time MDL.sample(iter=100000, burn=80000, burn_till_tuned=False, thin=2, progress_bar=True)

In [ ]:
pymc.Matplot.autocorrelation(MDL)

In [ ]:
pymc.Matplot.summary_plot(params)

In [ ]:
from corner import corner
from itertools import chain
data = np.vstack(chain([i.trace() for i in params])).T
corner(data, labels=[str(i) for i in params],
       quantiles=[0.16, 0.5, 0.84],
       truths=[150000, 10000, -1, -64024.38, 26.49419, 162397.3, -27.40575,
               -64024.38, 26.49419, 162397.3, -27.40575, -52440.88, 11.30117,
              -152397.3, 26.40575],
       show_titles=True, title_args={"fontsize": 40}).show()

In [ ]:
trace_dict = dict([(str(i), i.trace()) for i in params])
print(trace_dict)